# 同心云微博爬虫

### 思路：
多次下拉刷新新的微博，发现同心云用的显然是异步刷新，每次刷新能看到Network里面新的xhr资源，格式长这样：
http://yun.tongji.edu.cn/microblog/rest/microblog/getLastMicroBlogFromId?id=5ce3b208e4b0cbebede351ff&limit=100&forward=false&_=1558426627489
最后一串形如_=1558426627489的字串删掉之后发现没有影响，那就直接删去了。
然后是getLastMicroBlogFromId提示将按照Id来拉取微博，crtlF了一下直接看出这个是每次下拉之后刷出的第一条微博的Id。这个当然是可以修改的，于是为了达到爬取信息最新，开始找首条消息的Id。
这个其实也很好找，F5刷新之后Network里面的xhr是不包含这个getLastMicroBlogFromId的，也就是说一开始加载的时候页面里的20条是同步加载出来的。嗅探一下第一条微博，框架还是很明确的，有一个class="twt-type- mb-twt msg"可以拿来用（不久就发现这个是一般用户微博的class，系统管理员的还不一样，反正管理员的微博我不是很关心，也不太可能说刷出来的20条全是管理员发的）。于是BeautifulSoup去find一下id，直接把这个Id作为getLastMicroBlogFromId。这里后来遇到了一个问题，就是拿到的第一个Id放进去之后好像是无效的，我不能解释原因。当然也不在乎这一条，那就find_next从第二条开始，这就ok了。
forward=false是指时间倒序取资源，直观的看也就是取这条微博下方的微博。这个属性应该保留，不然...不然那还拉什么。
limit=20是指一次拉取20条微博，可以改动。
思路出来之后就是常规的套公式了。然后我一高兴忘了同心云要登陆才看得到微博，emmmmm，那就写吧。顺便提一句同心云直接静态表单就可以建立会话，这个简单得让人有一点失望。
当然数据出来是一个字串，直接切片去头去尾，json斩件成字典，挑出需要的数据。试着把limit改大一点，但太大的数据json解析不了，那就拿每次拉取的最后一个微博的id做索引，链式爬取，完事。
目前拉取的数据是["序号","id","姓名","正文","评论数","评论内容","点赞数","点赞人","收藏数","转发数","客户端型号","纬度","经度"]。评论内容在这个资源里只给出两条，其他需要/rest/comment/getComment?microBlogId=...通过另外的资源拿到，我比较懒没再去做了，他给多少我就拿多少。然后因为拿到的是网页信息，有一些html的<...>形式的元素理论上是可以通过RE去掉的，但又怕误删（其实还是懒）就没做。
还有一个问题是发现少数例子中点赞数与点赞人列表长度不匹配，这个我也没有去研究原理，直接按较小的输出。
最大的迷惑是拉取的数据也不是无限（相对意义）多的，只能拉到600条少一点点，然后就开始循环了。拿最后一条数据的id去用上面那个getLast了一下，发现不管limit再大，拿到的数据都只有那么几条。于是手动翻了一下网页，发现同心云居然是会刷到底的。我猜想是后台会定期删除数据的，所以要对抗的话还是得写一个定时爬取的版本，爬取间隔倒是不用太大。

In [ ]:
'''
Copyright @ Herobrine_1010 By 2019.5
'''
import requests
from bs4 import BeautifulSoup
import json
import time
import pandas as pd
import numpy as npy
url = "http://yun.tongji.edu.cn/space/c/rest/user/login"
UA = "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.13 Safari/537.36"
 
header = { "User-Agent" : UA,
           "Referer": "http://yun.tongji.edu.cn/public/login?supportNotMobile=false"
           }
 
txy_session = requests.Session()
f = txy_session.get(url,headers=header)
 
user_name=input("同心云登录手机号：")
psw1=input("同心云登陆密码：")
file_name=input("生成.csv的名字：")
postData = {
    'email':user_name,
    'password':psw1,
    'remember':'true',
    'accountType':'',
    'accountName':'',
    'networkId':'',
    'redirectUrl':'', 
    'forceToNetwork':'false'
}
 
txy_session.post(url,
                  data = postData,
                  headers = header)
 
f = txy_session.get('http://yun.tongji.edu.cn/microblog/',headers=header)

# print(f.content.decode())

soup = BeautifulSoup(f.content,"html.parser")
first = soup.find('div',{'class':'twt-type- mb-twt msg'})['id']
#print("第一条id: "+first)
first_element = soup.find('div',{'class':'twt-type- mb-twt msg'})
second=first_element.find_next('div',{'class':'twt-type- mb-twt msg'})['id']
#print("第二条id: "+second)

temp_id=second
# print(temp_id)
info_num=10

def by_id(weibo_id):
    global temp_id
    global l
    url='http://yun.tongji.edu.cn/microblog/rest/microblog/getLastMicroBlogFromId?id='+weibo_id+'&limit='+str(info_num)+'&forward=false'
    #print("url: "+url)
    f=txy_session.get(url,headers=header)
    contents=f.text

# def GetMiddleStr(content,startStr,endStr):
#     patternStr = r'%s(.+?)%s'%(startStr,endStr)
#     p = re.compile(patternStr,re.IGNORECASE)
#     m= re.match(p,content)
#     if m:
#         return m.group(1)

    contents=contents[17:len(contents)-91]

    text=json.loads(contents)

    all_list=[]
    title=["序号","id","姓名","正文","评论数","评论内容","点赞数","点赞人","收藏数","转发数","客户端型号","纬度","经度"]
    
    
    
    
    for i in range(0,info_num if info_num<=len(text) else len(text)):

        a=text[i]
        mylist=[]
        mylist.append(a["numericId"])
        mylist.append(a["id"])
        mylist.append(a["name"])
        mylist.append(a["content"])

        mylist.append(a["commentNumber"])
        commentlist=""


        if a["commentNumber"]!=0:
            try:
                for k in range(0,int(a["commentNumber"]) if int(a["commentNumber"])<2 else 2):
                            commentlist+=str(a["comments"][k]["name"])
                            commentlist+="："
                            commentlist+=a["comments"][k]["content"]
                            commentlist+="|"
    #             print(a['comments'][0]['name'])
    #     commentlist+="\b\b"
                mylist.append(commentlist)
            except:
                mylist.append(" ")
        else:
            mylist.append(" ")


        mylist.append(a["likeUserCount"])
        likelist=" "
        if a["likeUserCount"]>0:
            if a["likeUserCount"]<=60:
                for k in range(0,len(a["likeUsers"])):
                    likelist+=a["likeUsers"][k]["name"]
                    likelist+=","
            else:
                for k in range(0,59):
                    likelist+=a["likeUsers"][k]["name"]
                    likelist+=","
                likelist+="..."
    #     likelist+="\b"

        mylist.append(likelist)

        mylist.append(a["favoriteNumber"])
        mylist.append(a["forwardNumber"])
        mylist.append(a["from"])


        for key in a:
            if "location" in key:
                if str(a["locationData"]["latitude"])!='-1.0':
                    mylist.append(str(a["locationData"]["latitude"]))
                    mylist.append(str(a["locationData"]["longitude"]))


        if len(mylist)<12:
            mylist.append("未知")
            mylist.append("未知")
        all_list.append(mylist)

    dataframe = pd.DataFrame(all_list)
    dataframe.columns=title
    #print(dataframe)



    imananji=str(time.strftime('%Y_%m_%d_%H_%M_%S ',time.localtime(time.time())))


    dataframe.to_csv(str(file_name)+".csv", mode='a', index=False, sep=',', header=False, encoding="utf_8_sig" )
    #dataframe.to_csv("同心云_"+imananji+".csv", mode='a', index=False, sep=',', header=False, encoding="utf_8_sig" )
    temp_id=text[(info_num if info_num<=len(text) else len(text))-1]["id"]

    #return text[info_num-1]["id"]
# def circulate(initial_id,run_time):


l=[]
title=["序号","id","姓名","正文","评论数","评论内容","点赞数","点赞人","收藏数","转发数","客户端型号","纬度","经度"]
l.append(title)
dataframe = pd.DataFrame(l)
print(dataframe)
# dataframe.to_csv("同心云2.csv", mode='a', index=False, sep=',', header=False, encoding="utf_8_sig" )

for i in range(0,100):
    by_id(temp_id)
print("done")
